In [1]:
import intake
import xarray as xr
from matplotlib import pyplot as plt
import numpy as np
import xgcm
%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 6

In [2]:
from dask.distributed import Client, progress

from dask_kubernetes import KubeCluster
cluster = KubeCluster()
cluster.adapt(maximum=20)
client = Client(cluster)
cluster

distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:    tcp://10.32.5.27:32973
distributed.scheduler - INFO -   dashboard at:                     :8787
distributed.scheduler - INFO - Receive client connection: Client-88f0668a-110d-11ea-8176-ae04114d47ca
distributed.core - INFO - Starting established connection


In [19]:
import intake
col_url='https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json'
col = intake.open_esm_datastore(col_url)

In [26]:
import pprint
uni_dict = col.unique(['source_id', 'experiment_id', 'table_id'])
pprint.pprint(uni_dict, compact=True)

{'experiment_id': {'count': 99,
                   'values': ['piClim-CH4', 'piClim-NTCF', 'piClim-control',
                              'ssp370', 'piClim-2xDMS', 'piClim-2xdust',
                              'piClim-2xfire', 'piClim-2xss', 'piClim-BC',
                              'piClim-HC', 'piClim-N2O', 'piClim-OC',
                              'piClim-SO2', '1pctCO2-bgc', 'esm-ssp585',
                              'hist-bgc', 'amip-4xCO2', 'amip-future4K',
                              'amip-m4K', 'amip-p4K', 'amip', 'abrupt-2xCO2',
                              'abrupt-solp4p', 'abrupt-0p5xCO2', 'amip-lwoff',
                              'amip-p4K-lwoff', 'abrupt-solm4p', 'aqua-4xCO2',
                              'aqua-control-lwoff', 'aqua-control',
                              'aqua-p4K-lwoff', 'aqua-p4K', '1pctCO2',
                              'abrupt-4xCO2', 'historical', 'piControl',
                              'esm-hist', 'esm-piControl', 'ssp126', 'ssp245',


In [75]:
cat = col.search(source_id='UKESM1-0-LL',experiment_id=['ssp585'],table_id='Omon',variable_id=['so','thetao'],grid_label='gn')
cat.df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year
0,ScenarioMIP,MOHC,UKESM1-0-LL,ssp585,r1i1p1f2,Omon,so,gn,gs://cmip6/ScenarioMIP/MOHC/UKESM1-0-LL/ssp585...,NaN
1,ScenarioMIP,MOHC,UKESM1-0-LL,ssp585,r1i1p1f2,Omon,thetao,gn,gs://cmip6/ScenarioMIP/MOHC/UKESM1-0-LL/ssp585...,NaN


In [76]:
cat.df.institution_id

0    MOHC
1    MOHC
Name: institution_id, dtype: object

In [101]:
dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times': True},
                                cdf_kwargs={'chunks': {}, 'decode_times': True})

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)


In [102]:
dset_dict.keys()


dict_keys(['ScenarioMIP.MOHC.UKESM1-0-LL.ssp585.Omon.gn'])

In [103]:
ssp = dset_dict['ScenarioMIP.MOHC.UKESM1-0-LL.ssp585.Omon.gn']
ssp

<xarray.Dataset>
Dimensions:             (bnds: 2, i: 360, j: 330, lev: 75, member_id: 1, time: 1032, vertices: 4)
Coordinates:
    latitude            (j, i) float32 dask.array<chunksize=(330, 360), meta=np.ndarray>
    lev_bnds            (lev, bnds) float64 dask.array<chunksize=(75, 2), meta=np.ndarray>
    longitude           (j, i) float32 dask.array<chunksize=(330, 360), meta=np.ndarray>
    time_bnds           (time, bnds) object dask.array<chunksize=(1032, 2), meta=np.ndarray>
  * time                (time) object 2015-01-16 00:00:00 ... 2100-12-16 00:00:00
  * i                   (i) int32 0 1 2 3 4 5 6 ... 353 354 355 356 357 358 359
  * j                   (j) int32 0 1 2 3 4 5 6 ... 323 324 325 326 327 328 329
  * lev                 (lev) float64 0.5058 1.556 2.668 ... 5.698e+03 5.902e+03
  * member_id           (member_id) <U8 'r1i1p1f2'
Dimensions without coordinates: bnds, vertices
Data variables:
    vertices_latitude   (j, i, vertices) float32 dask.array<chunksize=(33

In [104]:
ssp['thetao']

<xarray.DataArray 'thetao' (member_id: 1, time: 1032, lev: 75, j: 330, i: 360)>
dask.array<broadcast_to, shape=(1, 1032, 75, 330, 360), dtype=float32, chunksize=(1, 12, 75, 330, 360), chunktype=numpy.ndarray>
Coordinates:
    latitude   (j, i) float32 dask.array<chunksize=(330, 360), meta=np.ndarray>
    longitude  (j, i) float32 dask.array<chunksize=(330, 360), meta=np.ndarray>
  * time       (time) object 2015-01-16 00:00:00 ... 2100-12-16 00:00:00
  * i          (i) int32 0 1 2 3 4 5 6 7 8 ... 352 353 354 355 356 357 358 359
  * j          (j) int32 0 1 2 3 4 5 6 7 8 ... 322 323 324 325 326 327 328 329
  * lev        (lev) float64 0.5058 1.556 2.668 ... 5.698e+03 5.902e+03
  * member_id  (member_id) <U8 'r1i1p1f2'
Attributes:
    cell_measures:  area: areacello volume: volcello
    cell_methods:   area: mean where sea time: mean
    comment:        mo: This variable is reported using a z* coordinate syste...
    long_name:      Sea Water Potential Temperature
    original_name:  mo:

In [179]:
import xesmf as xe
ssp=ssp.groupby('time.year').mean('time')

ssp = ssp.rename({'longitude': 'lon', 'latitude': 'lat'})
ds_out = xr.Dataset({'lat': (['lat'], np.arange(-90, 90, 1.0)),
                     'lon': (['lon'], np.arange(0, 360, 1.5))})
regridder = xe.Regridder(ssp, ds_out, 'bilinear')
ssp_regrid = regridder(ssp)



KeyboardInterrupt: 

In [174]:
meantheta_regrid=meantheta_regrid.sel(lat=slice(-90,-30))
meanso_regrid=meanso_regrid.sel(lat=slice(-90,-30))

In [175]:
meanso_regrid

<xarray.DataArray 'so' (member_id: 1, year: 86, lev: 75, lat: 61, lon: 240)>
dask.array<getitem, shape=(1, 86, 75, 61, 240), dtype=float64, chunksize=(1, 1, 75, 61, 240), chunktype=numpy.ndarray>
Coordinates:
  * lev        (lev) float64 0.5058 1.556 2.668 ... 5.698e+03 5.902e+03
  * member_id  (member_id) <U8 'r1i1p1f2'
  * year       (year) int64 2015 2016 2017 2018 2019 ... 2097 2098 2099 2100
  * lon        (lon) float64 0.0 1.5 3.0 4.5 6.0 ... 354.0 355.5 357.0 358.5
  * lat        (lat) float64 -90.0 -89.0 -88.0 -87.0 ... -33.0 -32.0 -31.0 -30.0
Attributes:
    regrid_method:  bilinear

In [ ]:
import seawater as csiro
pdens0 = csiro.dens(meanso_regrid,meantheta_regrid,0.)
